<a href="https://colab.research.google.com/github/mirnanoukari/FoR/blob/main/Assignment2/Alnoukari_Mirna_HA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fundamentals of Robotics**

#### *Assignment 2, Jacobian.*

##### Work of: Mirna Alnoukari


## **1. Robot Description**
Antropomorphic Arm with Spherical Wrist, the robot have 6 joints and is a 6 DoF manipulator.

The robot's configuration for Euler angles arrangement of $zyz$ is described with the figure below

<img src= "https://drive.google.com/uc?export=view&id=19VXCMEFHt67KQybZLwCOM66Kw0_ZFxO1" alt="inverse" width="500" />

***Figure 1: Robot model scheme***

<img src= "https://drive.google.com/uc?export=view&id=1B1xmxZCdJswWlRq4tN4b8ULqmDUpqWyH" alt="inverse" width="600" />

***Figure 2: zero configuration scheme***

## **2. Fixed Issues from Previous Assignment**


1.   Length of the last three parts of the robot (l4, l5, l6) were given values other than zeros' because it caused singularities at all values of q. 

2.   Edited inverse kinematics after giving lengths to translation. Also made some switching between translations when it is possible (same axes).


<img class="center" src="https://drive.google.com/uc?export=view&id=1obSEKg4FrJ3QeIYOTK6LGrPE-o-t1amm" alt="inverse" width="400" />

***Figure 3: side view of the manipulator***


In [ ]:
import numpy as np
from numpy.linalg import inv
import sympy as sp

def Rx(q):
   return np.array([[1, 0, 0, 0],
              [0, np.cos(q), - np.sin(q), 0],
              [0, np.sin(q), np.cos(q), 0],
              [0, 0, 0, 1]])
   

def Ry (q):
  return np.array([[np.cos(q), 0, np.sin(q), 0],
              [0, 1, 0, 0],
              [- np.sin(q), 0, np.cos(q), 0],
              [0, 0, 0, 1]])

def Rz(q):
  return np.array([[np.cos(q), - np.sin(q), 0, 0],
              [np.sin(q), np.cos(q), 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

def Tx(d):
   return np.array([[1, 0, 0, d],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
   
def Ty(d):
  return np.array([[1, 0, 0, 0],
              [0, 1, 0, d],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

def Tz(d):
  return np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, d],
              [0, 0, 0, 1]])

l = [1, 1, 1, 1, 1, 1]

def T1(q1):
  return Rz(q1) @ Tz(l[0])

def T2(q2):
  return Rx(q2) @ Tz(l[1])

def T3(q3):
  return Rx(q3) @ Tz(l[2])
  
def T4(q4):
  return Tz(l[3]) @ Rz(q4)

def T5(q5):
  return Ry(q5)

def T6(q6):
  return Rz(q6) @ Tz(l[4]) @ Tz(l[5])

def FK_solve(q, flag):
  A = [T1(q[0]), T2(q[1]), T3(q[2]), T4(q[3]), T5(q[4]), T6(q[5])]
  def f(A):
    ret = np.eye(4)
    for a in A:
      ret = ret @ a
    return ret

  if flag == 'ee': return f(A)
  if flag == 'full': return [f(A[:i+1]) for i in range(6)]

def transform_base(trans, q):
  return trans @ FK_solve(q, 'ee')
import itertools as it
#This is a generator that returns the possible solutions for q 
def IK_solve(base_frame, ee_frame):
  frame = inv(base_frame) @ ee_frame 
  #Tw is to get rid of the translations Tz(l[5]) and Tz(l[4]) that were added in T6 
  Tw = frame @ inv(Tz(l[5])) @ inv(Tz(l[4]))
  x, y, z = Tw[0, 3], Tw[1, 3], Tw[2, 3]
  # solving q1
  q1_sols = [-np.arctan2(x, y)]
  q1_sols.append(q1_sols[0] + np.pi)
  
  for q1 in q1_sols:
    #solving q2
    d = np.sqrt(x**2 + y**2)
    e = z - l[0]
    h = np.sqrt(d**2 + e**2)

    theta_1_sols = [np.arccos((l[1]**2 + h**2 - (l[2] + l[3])**2) / (2* l[1] * h))]
    theta_1_sols.append(-theta_1_sols[0])
    
    theta_2_sols = [np.arccos((l[0] ** 2 + h ** 2 - (x ** 2 + y ** 2 + z ** 2)) / (2 * l[0] * h))]
    theta_2_sols.append(-theta_2_sols[0])
    
    q2_sols = [np.pi - (theta_1 + theta_2) for theta_1, theta_2 in it.product(theta_1_sols, theta_2_sols)]
    
    for q2 in q2_sols:
      #solving q3
      q3_sols = [np.arccos(-(l[1]**2 + (l[2] + l[3])**2 - ((z - l[0])**2 + x**2 + y**2)) / (2 * l[1] * (l[2] + l[3])))]
      q3_sols.append(-q3_sols[0])
      
      for q3 in q3_sols:
        # solving q5
        T123 = T1(q1) @ T2(q2) @ T3(q3)
        # print(f'123: {T123}')
        T456 = inv(Tz(l[3])) @ inv(T123) @ Tw
        # print(f'456: {T456}')
        q5_sols = [np.arccos(T456[2, 2])]
        q5_sols.append(-q5_sols[0])
        for q5 in q5_sols:
          # solving q4
          if np.abs(q5) > 1e-3:
            q4_sols = [np.arcsin(T456[1, 2] / np.sin(q5))]
            q4_sols.append(np.pi - q4_sols[0])
          else:
            q4_sols = [0]
          for q4 in q4_sols:
            # solving q6
            if np.abs(q5) > 1e-3:
              q6_sols = [np.arccos(-T456[2, 0] / np.sin(q5))]
              q6_sols.append(-q6_sols[0])
            else:
              q6_sols = [np.arccos(T456[0, 0])]
              q6_sols.append(-q6_sols[0])
            for q6 in q6_sols:
              q = [q1, q2, q3, q4, q5, q6]
              
              if np.sum(np.abs(transform_base(base_frame, q) - ee_frame)) < 1e-5:
                yield q

for sol in IK_solve(np.eye(4), FK_solve([1, 2, 3, 4, 5, 6], 'ee')):
  print(sol)

[1.0, 1.9999999999999987, 2.9999999999999996, 0.8584073464102063, 1.2831853071795851, 2.8584073464102073]
[1.0, 1.9999999999999987, 2.9999999999999996, 3.9999999999999996, -1.2831853071795851, -0.2831853071795859]
[1.0, 7.722353955356057, -2.9999999999999996, 0.7800887828171531, 1.0795645970312828, 3.0630170278723474]
[1.0, 7.722353955356057, -2.9999999999999996, 3.921681436406946, -1.0795645970312828, -0.07857562571744596]
[4.141592653589793, -1.4391686481764712, 2.9999999999999996, 3.9216814364069466, 1.0795645970312828, 3.0630170278723474]
[4.141592653589793, -1.4391686481764712, 2.9999999999999996, 0.7800887828171533, -1.0795645970312828, -0.07857562571744596]
[4.141592653589793, 4.283185307179588, -2.9999999999999996, 3.9999999999999996, 1.2831853071795851, 2.858407346410207]
[4.141592653589793, 4.283185307179588, -2.9999999999999996, 0.8584073464102063, -1.2831853071795851, -0.2831853071795863]


In [ ]:
#Validation test
successes = 0
total = 0
for _ in range(10):
  q = 2 * np.pi * np.random.random(6)
  ee = FK_solve(q, 'ee')
  for iq in IK_solve(np.eye(4), ee):
    total += 1
    iee = FK_solve(iq, 'ee')
    if np.sum(np.abs(ee - iee)) > 1e-3:
      print('ERROR')
      print(q)
      print(ee)
      print(iee)
    else:
      successes += 1

print(f'{successes} out of {total} successes, {successes / total * 100}% accuracy')

80 out of 80 successes, 100.0% accuracy


## **3. Jacobian Matrix**
Jacobian matrix describes the relationship between every joint with the end effector; it relates linear and angular velocity of the end effector with joint velocities.

To calculate the Jacobian matrix, we need forward kinematics solution:

$$
T = T_{base}\cdot R_z^{q_1}\space T_z^{l_1} \cdot R_x^{q_2}\space T_z^{l_2} \cdot R_x^{q_3}\space T_z^{l_3} \cdot T_z^{l_4} \space R_z^{q_4} \cdot R_y^{q_5} \cdot R_z^{q_6} \space T_z^{l_5} \space T_z^{l_6} \cdot T_{tool}
$$


> Note: This part was edited when we decided to add $T_z^{l_4}$, $T_z^{l_5}$ and $T_z^{l_6}$ with some change in their order in case of same axis configuration, to have consequent rotation matrices.



We will use FK_solve(q, flag) function from the previous assignment.

The Jacobian of revolute joints is calculated using the following equation: 

$$
J_i = \left(
\begin{array}{cccc}
U_{i-1} \times (O_n - O_0) \\ 
U_{i-1}
\end{array}
\right)
$$

Where $U_i$ is the column vector corresponding to the axis of the rotation matrix, and $O_i$ is the vector of position from forward kinematics transformation. The Jacobian will be written at the following format:    

$$
J_i = \left(
\begin{array}{cccc}
 J_1 & J_2 & J_3 & J_4 & J_5 & J_6
\end{array}
\right)
$$

In [ ]:
def Jacobian(F):
  Fext = [np.eye(4)] + F
  
  u = [2, 0, 0, 2, 1, 2]
  U = [Fext[i][:3, u[i]] for i in range(6)]
  
  O = [Fext[i][:3, 3] for i in range(7)]

  J = np.zeros((6, 6))
  for i in range (6):
    J[:,i] = np.concatenate([np.cross(U[i], (O[6] - O[i])), U[i]])

  return J

## **4. Singularity Analysis**

Singularity cases can be obtained from the Jacobian matrix with different approaches as discussed in lecture, here we will use those two cases:

1.   Jacobian matrix determinant becomes zero
2.   Checking the rank of the Jacobian matrix 
 
We have found one case of singularity when $q_5 = 0$ only, with other random values for $q$'s.






In [ ]:
def Check_singular(Jacobian):
  return np.linalg.matrix_rank(Jacobian) < 6

Example of singularity in case $q_5 = 0$:

In [ ]:
q = np.array([2, 1, 4, 5, 0, 3])
print('det:', np.linalg.det(Jacobian(FK_solve(q, 'full'))))
print('Rank:', np.linalg.matrix_rank(Jacobian(FK_solve(q, 'full'))))
print('Jacobian \n', np.around(Jacobian(FK_solve(q, 'full')), 5))

det: 0.0
Rank: 5
Jacobian 
 [[ 1.24604  1.52303  1.03173 -0.       0.25859 -0.     ]
 [-2.72264  0.69703  0.47218  0.       0.74226  0.     ]
 [ 0.       2.99423  3.8357   0.       1.83907  0.     ]
 [ 0.      -0.41615 -0.41615 -0.87195 -0.47222 -0.87195]
 [ 0.       0.9093   0.9093  -0.39905  0.83846 -0.39905]
 [ 1.       0.       0.       0.28366 -0.27201  0.28366]]


## **5.  Cartesian Velocity**

The relation between joint velocities and cartesian velocities:

$$
𝑋 ̇ = J(q) \ q̇
$$

That means that in order to get the cartesian velocity, we only need to multiply Jacobian matrix we already obtained with $q̇$.

In [ ]:
def cartesian_vel(Jacobian, q_dot):
  return Jacobian @ q_dot

## **Numerical Solution**

Using the definition of differentiations, we implemented the following equation to calculate Jacobian:    


```
𝐽 = FK_solve(q2, 'ee') - FK_solve(q, 'ee') / ɛ

```

Where $ε < 10^{-3}$
 


In [ ]:
q2 = np.zeros(5+1)
eps = 0.001
q2[0] += np.pi/4 + eps
print(q)
print(q2)
fkq1 = FK_solve(q, 'ee')
fkq2 = FK_solve(q2, 'ee')
print(fkq1)
print(fkq2)
delta = np.subtract(fkq2 , fkq1) / eps
print(delta)

[2 1 4 5 0 3]
[0.78639816 0.         0.         0.         0.         0.        ]
[[-0.19463905  0.44924761 -0.87194738 -2.7226421 ]
 [-0.24909172 -0.88244534 -0.3990533  -1.24603773]
 [-0.94871964  0.13952351  0.28366219  2.67495105]
 [ 0.          0.          0.          1.        ]]
[[ 0.70639932 -0.70781353  0.          0.        ]
 [ 0.70781353  0.70639932  0.          0.        ]
 [ 0.          0.          1.          6.        ]
 [ 0.          0.          0.          1.        ]]
[[  901.038375   -1157.06114203   871.94737547  2722.64210065]
 [  956.9052547   1588.84465972   399.05330339  1246.03772518]
 [  948.71963903  -139.52351438   716.33781454  3325.04895228]
 [    0.             0.             0.             0.        ]]
